In [34]:
import datetime
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [12]:
portal_client = get_portal_client()

In [28]:
users_df = portal_client.query_to_pandas(
    """SELECT
        u.id,
        user_level_id,
        u.email,
        NOT ISNULL(service_users.user_id) AS serviced,
        created_at
    FROM topaz.api_users AS u
    LEFT JOIN (
        SELECT DISTINCT user_id
        FROM topaz.api_visits
        UNION
        SELECT DISTINCT user_id
        FROM topaz.api_mode_changes
        WHERE mode IN ("S", "L")
        UNION
        SELECT DISTINCT user_id
        FROM topaz.api_service_tasks_record
    ) AS service_users
    ON u.id = service_users.user_id
    WHERE u.user_level_id IN (5, 6)
    AND u.deleted_at IS NULL"""
)

In [ ]:
engineer_mask = users_df["user_level_id"] == 6
serviced_mask = users_df["serviced"] == 1
june_onwards_mask = users_df["created_at"] >= datetime.datetime(2025,6,1)

In [39]:
problem_users = users_df.loc[engineer_mask & ~serviced_mask & june_onwards_mask, "id"].to_list()

In [42]:
placeholders = ', '.join(['%s'] * len(problem_users))
problem_users_df = portal_client.query_to_pandas(
    f"""WITH RECURSIVE category_path AS (
    SELECT id, name, id AS root_id, name AS root_name
    FROM api_clients
    WHERE parent_id = 1

    UNION ALL

    SELECT c.id, c.name, cp.root_id, cp.root_name
    FROM api_clients c
    INNER JOIN category_path cp ON cp.id = c.parent_id
    )
    SELECT
        u.id AS user_id,
        u.name AS user_name,
        cp.root_id,
        cp.root_name,
        SUBSTRING_INDEX(email, '@', -1) AS domain
    FROM topaz.api_users AS u
    INNER JOIN topaz.api_collaborators AS c
    ON u.id = c.user_id
    INNER JOIN category_path AS cp
    ON c.client_id = cp.id 
    WHERE u.id IN ({placeholders})
    AND u.deleted_at IS NULL
    AND c.deleted_at IS NULL""",
    params=problem_users
)

In [47]:
problem_users_df.groupby(["root_name", "domain"]).count().iloc[:50]

user_id  \
root_name                               domain                                
4 Site Security Systems Ltd             4sitesecurity.co.uk               4   
ADT Fire & Security PLC                 crieffhydro.com                   1   
                                        jci.com                           2   
AKD Fire & Security Ltd                 akdfiresecurity.co.uk             1   
ARC Fire Safety & Security Ltd          arcfiresafety.co.uk               1   
Absolute Fire & Security Systems Ltd    absolutesecurity.co.uk            1   
                                        acl.uk.com                        1   
Albion Detection Systems Ltd            albion-detection.co.uk            1   
Allsaved Ltd                            allsaveduk.com                    2   
Ampac Pty Limited                       platinumfire.com.au               1   
Artisan Fire and Security Ltd           metrosecurity.co.uk               1   
Ascot Services UK Ltd                   ascotservices.net                 2   
Avansys LLP                             fixfire.co.uk                     1   
Bell Fire & Security Ltd                bellfireandsecurity.com           2   
C.I. Fire & Security Ltd                cifire.je                         4   
CSS Systems Ltd                         csssystems.co.uk                  1   
Cannon Fire & Security                  cannonsecurity.co.uk              4   
                                        gmail.com                         1   
Chubb Fire & Security Ltd               chubbfs.com                       2   
Churches Fire Security Ltd              churchesfire.com                 17   
Click 24 Ltd                            click24.co.uk                     1   
Clyde & Solway Systems Ltd              candssystems.co.uk                1   
Crafted Technologies Limited            craftedtechnologies.co.uk         2   
D2 Integrated Systems Ltd               d2is.co.uk                        1   
DPJ Electrical Services Limited         dpjfireandsecurity.co.uk          1   
Drax 360 (UK) Ltd                       drax360.com                       2   
EA-RS Fire Engineering Ltd              ea-rsfire.com                     1   
EFT Systems Ltd                         eftsystems.co.uk                  1   
                                        hotmail.com                       1   
Eurotech Fire Systems Ltd               nomicoelectrical.co.uk            1   
Exitex Ltd                              sfpsni.co.uk                      1   
Falcon Security Systems Ltd             falcsec.com                       5   
Firesec Compliance Limited              firesec.uk.com                    1   
Freedom Fire & Security                 freedomfsm.co.uk                  1   
Gentoo Group Ltd                        gentoogroup.com                   3   
Griffin & General Fire Services Limited griffinfire.co.uk                 1   
Hall & Kay Fire Engineering             hkfire.co.uk                      3   
                                        hotmail.co.uk                     1   
I.C. Service & Maintenance UK Ltd       icservice.uk.com                  4   
Ideal Fire Limited                      idealfire.ie                      1   
Imperial Fire & Security Ltd            ifs.ltd                           7   
Islington Council                       vincifacilities.com               2   
JMS Fire and Security Limited           bucksfireandsecurity.co.uk        1   
KA Facilities Management Ltd            kafm.co.uk                        1   
KASA-Tec Limited                        kasa-tec.co.uk                    1   
Leader Systems (UK) Limited             leader-systems.co.uk              2   
M D Lawrence Limited                    mdl-group.com                     1   
Marlowe Fire & Security                 marlowefireandsecurity.com       22   
                                        nimbusdigital.com                 1   
McElwaine Security Services Ltd        

In [ ]:
users_df = portal_client.query_to_pandas(
    """SELECT
        u.id,
        user_level_id,
        u.email,
        NOT ISNULL(service_users.user_id) AS serviced,
        created_at
    FROM topaz.api_users AS u
    LEFT JOIN (
        SELECT DISTINCT user_id
        FROM topaz.api_visits
        UNION
        SELECT DISTINCT user_id
        FROM topaz.api_mode_changes
        WHERE mode IN ("S", "L")
        UNION
        SELECT DISTINCT user_id
        FROM topaz.api_service_tasks_record
    ) AS service_users
    ON u.id = service_users.user_id
    WHERE u.user_level_id IN (5, 6)
    AND u.deleted_at IS NULL"""
)

[19326,
 19355,
 19375,
 19414,
 19419,
 19420,
 19421,
 19422,
 19437,
 19438,
 19446,
 19448,
 19451,
 19453,
 19454,
 19455,
 19456,
 19457,
 19460,
 19468,
 19469,
 19481,
 19494,
 19498,
 19501,
 19502,
 19504,
 19509,
 19536,
 19541,
 19558,
 19580,
 19581,
 19582,
 19583,
 19585,
 19586,
 19587,
 19589,
 19595,
 19602,
 19603,
 19605,
 19606,
 19613,
 19614,
 19640,
 19645,
 19649,
 19652,
 19653,
 19654,
 19655,
 19656,
 19657,
 19660,
 19661,
 19663,
 19672,
 19714,
 19715,
 19718,
 19719,
 19720,
 19731,
 19732,
 19733,
 19734,
 19735,
 19736,
 19737,
 19738,
 19744,
 19746,
 19747,
 19749,
 19751,
 19752,
 19753,
 19754,
 19755,
 19757,
 19758,
 19770,
 19771,
 19774,
 19775,
 19776,
 19777,
 19778,
 19779,
 19780,
 19781,
 19782,
 19783,
 19784,
 19785,
 19794,
 19795,
 19797,
 19798,
 19799,
 19800,
 19801,
 19802,
 19803,
 19804,
 19808,
 19809,
 19828,
 19830,
 19836,
 19837,
 19839,
 19840,
 19842,
 19851,
 19852,
 19854,
 19855,
 19856,
 19857,
 19859,
 19862,
 19863,


In [3]:
end_users_df = portal_client.query_to_pandas(
    """SELECT
        *,
        EXISTS (SELECT 1 FROM topaz.api_service_engineers WHERE user_id = u.id) AS service_completed
    FROM topaz.api_users AS u
    WHERE u.user_level_id = 5
    AND u.deleted_at IS NULL"""
)
serviced_mask = end_users_df["service_completed"] == 1
end_users_w_service = end_users_df.loc[serviced_mask, "id"].to_list()

In [4]:
engineers_df = portal_client.query_to_pandas(
    """SELECT
        *,
        EXISTS (SELECT 1 FROM topaz.api_service_engineers WHERE user_id = u.id) AS service_completed
    FROM topaz.api_users AS u
    WHERE u.user_level_id = 6
    AND u.deleted_at IS NULL"""
)
unserviced_mask = engineers_df["service_completed"] == 0
engineers_no_service = engineers_df.loc[unserviced_mask, "id"].to_list()

In [5]:
engineers_df[unserviced_mask]

,id,urlkey,user_level_id,language_id,timezone_id,name,email,phone,company_name,company_address,...,error,created_at,updated_at,active_at,onholiday_til,deleted_at,test_sent_at,test_received_at,gcm_id,service_completed
0,52,q0xk8emk8m,6,1,36,Paul Norman,paulnorman@minervafire.com,07971 463658,None,None,...,(81.106.48.31:57288) validation failed (Certi...,2014-09-09 10:26:40,2023-03-06 22:26:25,2015-08-24 10:46:47,NaT,None,NaT,NaT,None,0
1,57,ez_ko10$0l,6,1,36,Lee Frettingham,leefrettingham@marlowefireandsecurity.com,07968765919,None,None,...,(81.106.48.31:57288) validation failed (Certi...,2014-09-09 13:51:00,2024-07-24 11:10:12,2024-07-24 11:10:12,NaT,None,NaT,NaT,None,0
4,67,g3rkzmowx4,6,1,36,Andy Salisbury,andysalisbury@minervafire.com,07505 333437,None,None,...,(81.106.48.31:57288) validation failed (Certi...,2014-09-15 08:04:21,2023-03-06 22:26:25,2016-09-21 08:17:06,NaT,None,NaT,NaT,None,0
5,68,p4nky0vw58,6,1,36,Alistair Ogden (Accounts),alistairogden@minervafire.com,07974 570742,None,None,...,(81.106.48.31:57288) validation failed (Certi...,2014-09-15 08:05:36,2023-03-06 22:26:25,2016-10-03 14:37:53,NaT,None,NaT,NaT,None,0
6,69,qv-k9py$30,6,1,36,Tony Oliver,t.oliver@scotshield.com,07866980364,None,None,...,(81.106.48.31:57288) validation failed (Certi...,2014-09-17 10:32:24,2023-03-06 22:26:25,2015-11-24 10:14:38,NaT,None,NaT,NaT,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6577,20141,3r$z7p43kx,6,1,36,James,James@smith.co.uk,,None,None,...,None,2025-07-28 10:55:28,2025-07-28 10:55:28,NaT,NaT,None,NaT,NaT,None,0
6578,20144,v4kpor88wo,6,1,36,Ryan Chisholm,Ryan@rs-fands.co.uk,,None,None,...,None,2025-07-28 11:52:13,2025-07-28 11:52:13,NaT,NaT,None,NaT,NaT,None,0
6579,20145,n9w2jgp6$r,6,1,36,Chris Rudd,Chris@cifire.je,,None,None,...,None,2025-07-28 12:14:00,2025-07-28 13:16:32,2025-07-28 13:16:32,NaT,None,NaT,NaT,None,0
6580,20146,pjk541-pw7,6,1,36,Summer Boulter,summer@cifire.je,,None,None,...,None,2025-07-28 12:14:57,2025-07-28 12:14:57,NaT,NaT,None,NaT,NaT,None,0
